### purpose

fit GF to common garden climates


### notes

this notebook follows MVP_02_fit_gradient_forests, headers are used as landmarks to those in the mvp02 script

In [1]:
from pythonimports import *

import MVP_summary_functions as mvp
import MVP_02_fit_gradient_forests as mvp02

mvp.latest_commit()
session_info.show()

#########################################################
Today:	March 16, 2023 - 10:50:57
python version: 3.8.5

Current commit of pythonimports:
commit 8d4f83fd91caa2e58acbafcadfde07d19ae79379  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Fri Mar 3 00:48:36 2023 -0500

Current commit of MVP_offsets:
commit 11296c5e9cb778ba1aebb1c2ff0e5f499bf6f646  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Mon Mar 13 14:39:28 2023 -0400
#########################################################



# if __name__ == '__main__'

### input args

In [2]:
mvp02.seed = 'tutorial'

In [3]:
mvp02.rscript_exe = '/home/b.lind/anaconda3/envs/r35/lib/R/bin/Rscript'

In [4]:
mvp02.training_outdir = '/home/b.lind/offsets/run_20220919_tutorial/gradient_forests/training/training_outfiles'

### start cluster

In [5]:
lview, dview = get_client(cluster_id='1678978089-wyp6', profile='lotterhos')

16 16


### load objects to cluster

In [6]:
dview['rscript_exe'] = mvp02.rscript_exe
mvp02.fitting_file = '/home/b.lind/code/MVP-offsets/01_src/MVP_gf_fitting_script.R'
dview['fitting_file'] = mvp02.fitting_file

mvp02.lview = lview  # for mvp02.run_fit_gradient_forests

### create dirs

In [7]:
garden_dir, mvp02.fitting_dir, training_filedir = mvp02.make_fitting_dirs(mvp02.training_outdir)

mvp02.garden_dir = garden_dir


Creating directories ...
	gf_directory = '/home/b.lind/offsets/run_20220919_tutorial/gradient_forests'
	fitting_dir = '/home/b.lind/offsets/run_20220919_tutorial/gradient_forests/fitting/fitting_outfiles'
	garden_dir = '/home/b.lind/offsets/run_20220919_tutorial/gradient_forests/fitting/garden_files'
	training_filedir = '/home/b.lind/offsets/run_20220919_tutorial/gradient_forests/training/training_files'


# main

### get environmental data

In [8]:
# get_envdata (skip part of function that asserts two envfiles, one for ind and one for pooled)
files = fs(training_filedir, pattern='_envfile_')

envdfs = {}
envfiles = {}
for f in files:
    ind_or_pooled = op.basename(f).split("_")[-1].rstrip(".txt")
    envdfs[ind_or_pooled] = pd.read_table(f, index_col=0)
    envfiles[ind_or_pooled] = f

In [9]:
envfiles

{'pooled': '/home/b.lind/offsets/run_20220919_tutorial/gradient_forests/training/training_files/tutorial_envfile_GFready_pooled.txt'}

In [10]:
envdfs['pooled']

,env1_mat,env2_MTWetQ,env3_MTDQ,env4_PDM,env5_PwarmQ,env6_PWM
1,0.186225,-0.226935,0.861531,0.221974,-0.363974,0.761569
2,0.070353,-0.265649,0.799727,0.169512,-0.417965,0.315295
3,0.264633,-0.248372,0.375248,-0.172690,-0.586408,-0.320803
4,0.371616,0.167485,0.063118,-0.465978,-0.656053,-0.625240
5,0.244613,0.117437,-0.010939,-0.291486,-0.528748,-0.537376
...,...,...,...,...,...,...
96,-0.288220,0.261489,-0.317123,-0.018548,0.228030,-0.045559
97,-0.116570,0.323553,-0.231351,-0.269762,0.344912,0.059459
98,0.093915,0.400397,-0.150975,-0.408936,0.588880,0.274592
99,0.034177,0.399089,-0.440848,-0.464035,0.673533,0.360201


### create future climates (transplant climates) for each subpopID

In [11]:
garden_files = mvp02.create_garden_files(envdfs, envfiles)

len(garden_files['pooled'])


Creating "future climates" for each common garden ...


100

In [12]:
garden_files['pooled'][0]

'/home/b.lind/offsets/run_20220919_tutorial/gradient_forests/fitting/garden_files/tutorial_envfile_GFready_pooled_1.txt'

### parallelize fit of GF models to climates of each common garden (transplant location aka future climate)

In [13]:
mvp02.run_fit_gradient_forests(garden_files[:0])


Fitting gradient forest models to common garden climates ...

Watching 300 jobs ...


fitting gradient forests: 100%|███████████████| 300/300 [08:15<00:00,  1.65s/it]


Redoing any jobs that died ...


In [15]:
pd.read_table(garden_files['pooled'][0], index_col=0)

,env1_mat,env2_MTWetQ,env3_MTDQ,env4_PDM,env5_PwarmQ,env6_PWM
1,0.186225,-0.226935,0.861531,0.221974,-0.363974,0.761569
2,0.186225,-0.226935,0.861531,0.221974,-0.363974,0.761569
3,0.186225,-0.226935,0.861531,0.221974,-0.363974,0.761569
4,0.186225,-0.226935,0.861531,0.221974,-0.363974,0.761569
5,0.186225,-0.226935,0.861531,0.221974,-0.363974,0.761569
...,...,...,...,...,...,...
96,0.186225,-0.226935,0.861531,0.221974,-0.363974,0.761569
97,0.186225,-0.226935,0.861531,0.221974,-0.363974,0.761569
98,0.186225,-0.226935,0.861531,0.221974,-0.363974,0.761569
99,0.186225,-0.226935,0.861531,0.221974,-0.363974,0.761569
